# Decomposing Circuits in OpenSquirrel

Not all gates, such as the Hadamard gate, can be applied to physical qubits with a single operation. This is because each gate-based quantum computer has its own set of devices and architectural restrictions, which confine the _pulses_, or the operation used to create a gate on a quantum computer, to specific rotations, angles, and durations. 

Decomposition schemes extend a robust gate into a set of gates that can be pulsed on a quantum computer. OpenSquirrel offers many decomposition schemes that can be applied to a gate or a circuit.

## 1 Single Qubit Gates

### 1.1 ABA Decomposition

The ABA decomposition (or KAK decomposition) is a decomposition method that uses three consecutive Pauli rotation gates to form any unitary gate. Thus, any arbitrary single qubit gate can be expressed as a product of rotations: a rotation around the A axis, followed by a rotation around the B axis, and then another rotation around the A axis. The mathematical formalism for this rotation can be seen in section 3.1.

In OpenSquirrel, this can be found in the `get_decomposition_angles` method from the  `opensquirrel.decomposer.aba_decomposer` module. The ABA Decomposition can be applied to list of gates or a circuit. Below we will first focus on decomposing a circuit.

The circuit chosen is a single qubit circuit with an added Hadamard, Z, Y, and Rx gate. 

```
                   ┌───┐┌───┐┌───┐┌─────────┐
                q: ┤ H ├┤ Z ├┤ Y ├┤ Rx(π/3) ├
                   └───┘└───┘└───┘└─────────┘
```

This can be seen below, 

In [16]:
import math

from opensquirrel.circuit_builder import CircuitBuilder

# Build the circuit structure using the CircuitBuilder
builder = CircuitBuilder(qubit_register_size=1)
builder.H(0)
builder.Z(0)
builder.Y(0)
builder.Rx(0, math.pi / 3)

# Create a new circuit from the constructed structure
circuit = builder.to_circuit()
circuit

version 3.0

qubit[1] q

H q[0]
Z q[0]
Y q[0]
Rx(1.0471976) q[0]

Above we can see the current gates in our circuit. Having created a circuit, we can now use an ABA decomposition in `opensquirrel.decomposer.aba_decomposer` to decompose the gates in the circuit. For this example, we will apply the Z-Y-Z decomposition using the `ZYZDecomposer`. 

In [17]:
from opensquirrel.decomposer.aba_decomposer import ZYZDecomposer

# Decompose the circuit using ZYZDecomposer
circuit.decompose(decomposer=ZYZDecomposer())
circuit

version 3.0

qubit[1] q

Rz(3.1415927) q[0]
Ry(1.5707963) q[0]
Rz(3.1415927) q[0]
Ry(3.1415927) q[0]
Rz(1.5707963) q[0]
Ry(1.0471976) q[0]
Rz(-1.5707963) q[0]

```
       ┌───────┐┌─────────┐┌───────┐┌───────┐┌─────────┐┌────────────┐┌──────────┐
    q: ┤ Rz(π) ├┤ Ry(π/2) ├┤ Rz(π) ├┤ Ry(π) ├┤ Rz(π/2) ├┤ Ry(1.0472) ├┤ Rz(-π/2) ├
       └───────┘└─────────┘└───────┘└───────┘└─────────┘└────────────┘└──────────┘
```

All the gates are now a combination of rotations on the Z or Y axis. The first three gates are the decomposition of the Hadamard gate. Keep in mind that not every gate has to be decomposed into three gates! 

This can be seen with the Y and Z gates, where only a single rotation gate was necessary for each gate decomposition. The Pauli Y gate is equal to a rotation of $\pi$ radians on the Bloch sphere in the Y-axis while the Pauli Z is equal to a $\pi/2$ rotation on the Bloch sphere in the Z-axis.

Finally, the $\pi/3$ radian rotation in the X-axis can be decomposed into the last two remaining gates.

It is also possible to decompose individual gates. This can be seen below with the Hadamard gate.



In [18]:
from opensquirrel.decomposer.aba_decomposer import XZXDecomposer
from opensquirrel.default_gates import H

XZXDecomposer().decompose(H(0))

[BlochSphereRotation(Qubit[0], axis=Axis[1. 0. 0.], angle=1.5707963267948966, phase=0.0),
 BlochSphereRotation(Qubit[0], axis=Axis[0. 0. 1.], angle=1.5707963267948966, phase=0.0),
 BlochSphereRotation(Qubit[0], axis=Axis[1. 0. 0.], angle=1.5707963267948966, phase=0.0)]

### 1.2 McKay Decomposition

The McKay decomposition comes from this paper https://arxiv.org/pdf/1612.00858. 

This decomposition utilizes at least three arbitrary Z rotations and two X90 gates (90-degree rotations around the X-axis on the Bloch sphere), totaling a maximum of five gates. It is particularly powerful in experimental settings where Rz gates can be efficiently pulsed to small angles, while other rotations cannot. The mathematical formalism of the McKay decomposition can be found in section 3.2.

In OpenSquirrel, the `McKayDecomposer` works similarly to the `ABADecomposer`. Thus, we will begin once again by building a circuit the following circuit,
```
               ┌───┐┌───┐┌───┐┌─────────┐
            q: ┤ H ├┤ Z ├┤ X ├┤ Rx(π/3) ├
               └───┘└───┘└───┘└─────────┘
``` 

In [19]:
# Build the circuit structure using the CircuitBuilder
builder = CircuitBuilder(qubit_register_size=1)
builder.H(0)
builder.Z(0)
builder.X(0)
builder.Rx(0, math.pi / 3)

# Create a new circuit from the constructed structure
circuit = builder.to_circuit()
circuit

version 3.0

qubit[1] q

H q[0]
Z q[0]
X q[0]
Rx(1.0471976) q[0]

The `McKayDecomposer` is called from `opensquirrel.decomposer.mckay_decomposer` and used in a similar method to the `ABADecomposer`.

In [20]:
from opensquirrel.decomposer.mckay_decomposer import McKayDecomposer

# Decompose the circuit using ZYZDecomposer
circuit.decompose(decomposer=McKayDecomposer())
circuit

version 3.0

qubit[1] q

Rz(1.5707963) q[0]
X90 q[0]
Rz(1.5707963) q[0]
Rz(3.1415927) q[0]
X90 q[0]
X90 q[0]
Rz(-1.5707963) q[0]
X90 q[0]
Rz(2.0943951) q[0]
X90 q[0]
Rz(-1.5707963) q[0]

```
   ┌─────────┐┌─────────┐┌─────────┐┌───────┐┌─────────┐┌─────────┐┌──────────┐»
q: ┤ Rz(π/2) ├┤ Rx(π/4) ├┤ Rz(π/2) ├┤ Rz(π) ├┤ Rx(π/4) ├┤ Rx(π/4) ├┤ Rz(-π/2) ├»
   └─────────┘└─────────┘└─────────┘└───────┘└─────────┘└─────────┘└──────────┘»
«   ┌─────────┐┌────────────┐┌─────────┐┌──────────┐
«q: ┤ Rx(π/4) ├┤ Rz(2.0944) ├┤ Rx(π/4) ├┤ Rz(-π/2) ├
«   └─────────┘└────────────┘└─────────┘└──────────┘
```

Above you can see the decomposition. Depending on the gate, not all five gates are necessary for the decomposition as seen by the first three gates, which are used to decompose the Hadamard gate. 

The Pauli Z is simply a $\pi$ rotation around the Z-axis and the Pauli X gate is two consecutive X90 gates. The $\pi/3$ X axis rotation gate does subsequently use all five gates in it's decomposition. 

## 2. Multi-qubit Gates

### 2.1 ABC Decomposition

We can envision multi-qubit gates as controlled gates. This means that if a control qubit or qubit sets are equal to one, perform an operation on a target qubit (or qubits). Again, most architectures do not possess the full control capabilities of a controlled operation. Therefore, a multi-qubit gate is decomposed by three single qubit gates of arbitrary rotations (here we will use the `Rz` and `Ry`), with a `CNOT` gate in between each single qubit gate. An extra `Rz` rotation is added to the control qubit to preserve the global phase. This can be seen in section 3.3.

This decomposition scheme is referred to as the `cnot_decomposer` on OpenSquirrel. In order to test this, we will build a circuit with multi-qubit gates. Below is an example containing a two qubit circuit with a controlled Z gate between the first qubit (control) and the second qubit (target), a controlled Z-axis rotation of $\pi/3$ radians between the first qubit (control) and the second qubit (target) and an additional controlled Z-axis rotation of $\pi/2$ radians between the second qubit (control) and the first qubit (target).

```
                               ┌─────────┐
            q_0: ─■──────■─────┤ Rz(π/2) ├
                  │ ┌────┴────┐└────┬────┘
            q_1: ─■─┤ Rz(π/3) ├─────■─────
                    └─────────┘           
```


In [21]:
# Build the circuit structure using the CircuitBuilder
builder = CircuitBuilder(qubit_register_size=2)
builder.CZ(0, 1)
builder.CR(0, 1, math.pi / 3)
builder.CR(1, 0, math.pi / 2)

# Create a new circuit from the constructed structure
circuit = builder.to_circuit()
circuit

version 3.0

qubit[2] q

CZ q[0], q[1]
CR(1.0471976) q[0], q[1]
CR(1.5707963) q[1], q[0]

We can import `CNOTDecomposer` from `opensquirrel.decomposer.cnot_decomposer`. The above circuit can then be decomposed using `CNOTDecomposer` as seen below.

In [22]:
from opensquirrel.decomposer.cnot_decomposer import CNOTDecomposer

circuit.decompose(decomposer=CNOTDecomposer())
circuit

version 3.0

qubit[2] q

Rz(-3.1415927) q[1]
Ry(1.5707963) q[1]
CNOT q[0], q[1]
Ry(-1.5707963) q[1]
Rz(3.1415927) q[1]
Rz(0.52359878) q[1]
CNOT q[0], q[1]
Rz(-0.52359878) q[1]
CNOT q[0], q[1]
Rz(0.52359878) q[0]
Rz(0.78539816) q[0]
CNOT q[1], q[0]
Rz(-0.78539816) q[0]
CNOT q[1], q[0]
Rz(0.78539816) q[1]

```
                                                                       »
q_0: ───────────────────────■───────────────────────────────────────■──»
     ┌────────┐┌─────────┐┌─┴─┐┌──────────┐┌───────┐┌────────────┐┌─┴─┐»
q_1: ┤ Rz(-π) ├┤ Ry(π/2) ├┤ X ├┤ Ry(-π/2) ├┤ Rz(π) ├┤ Rz(0.5236) ├┤ X ├»
     └────────┘└─────────┘└───┘└──────────┘└───────┘└────────────┘└───┘»
«      ┌────────────┐┌────────────┐┌───┐┌─────────────┐┌───┐              
«q_0: ─┤ Rz(0.5236) ├┤ Rz(0.7854) ├┤ X ├┤ Rz(-0.7854) ├┤ X ├──────────────
«     ┌┴────────────┤└────────────┘└─┬─┘└─────────────┘└─┬─┘┌────────────┐
«q_1: ┤ Rz(-0.5236) ├────────────────■───────────────────■──┤ Rz(0.7854) ├
«     └─────────────┘                                       └────────────┘
```

## 3. Decomposition Theory

### 3.1 ABADecomposer
Let us mathematically look at one of the most common ABA decompositions, the Z-Y-Z decomposition.
A 3D rotation of angle $\alpha$ about the (normalized) axis $n=\left(n_x, n_y, n_z \right)$ can be represented by the quaternion $q = \cos\left(\alpha/2\right) + \sin\left(\alpha/2\right) \left( n_x i + n_y j + n_z k \right)$.
Since composition of rotations is equivalent of the product of their quaternions, we have to find angles $\theta_1$, $\theta_2$ and $\theta_3$ such that

$$
q =
\left[ \cos\left(\frac{\theta_1}{2}\right) + k \sin\left(\frac{\theta_1}{2}\right) \right]
\left[ \cos\left(\frac{\theta_2}{2}\right) + j \sin\left(\frac{\theta_2}{2}\right) \right]
\left[ \cos\left(\frac{\theta_3}{2}\right) + k \sin\left(\frac{\theta_3}{2}\right) \right]\ .
$$

Let us expand this last term with Sympy:



In [23]:
import sympy as sp

theta1, theta2, theta3 = sp.symbols("theta_1 theta_2 theta_3")

z1 = sp.algebras.Quaternion.from_axis_angle((0, 0, 1), theta1)
y = sp.algebras.Quaternion.from_axis_angle((0, 1, 0), theta2)
z2 = sp.algebras.Quaternion.from_axis_angle((0, 0, 1), theta3)

rhs = sp.trigsimp(sp.expand(z1 * y * z2))
rhs

cos(theta_2/2)*cos((theta_1 + theta_3)/2) + (-sin(theta_2/2)*sin((theta_1 - theta_3)/2))*i + sin(theta_2/2)*cos((theta_1 - theta_3)/2)*j + sin((theta_1 + theta_3)/2)*cos(theta_2/2)*k

Let us change variables and define $p\equiv\theta_1 + \theta_3$ and $m\equiv\theta_1 - \theta_3$.

In [24]:
p, m = sp.symbols("p m")

rhs_simplified = rhs.subs({theta1 + theta3: p, theta1 - theta3: m})

rhs_simplified

cos(p/2)*cos(theta_2/2) + (-sin(m/2)*sin(theta_2/2))*i + sin(theta_2/2)*cos(m/2)*j + sin(p/2)*cos(theta_2/2)*k

The original rotation's quaternion $q$ can be defined in Sympy accordingly:

In [25]:
alpha, nx, ny, nz = sp.symbols("alpha n_x n_y n_z")

q = sp.algebras.Quaternion.from_axis_angle((nx, ny, nz), alpha).subs(
    {nx**2 + ny**2 + nz**2: 1}  # We assume the axis is normalized.
)
q

cos(alpha/2) + n_x*sin(alpha/2)*i + n_y*sin(alpha/2)*j + n_z*sin(alpha/2)*k

We get the following system of equations, where the unknowns are $p$, $m$, and $\theta_2$:


In [26]:
from IPython.display import display

display(
    sp.Eq(rhs_simplified.a, q.a),
    sp.Eq(rhs_simplified.b, q.b),
    sp.Eq(rhs_simplified.c, q.c),
    sp.Eq(rhs_simplified.d, q.d),
)

Eq(cos(p/2)*cos(theta_2/2), cos(alpha/2))

Eq(-sin(m/2)*sin(theta_2/2), n_x*sin(alpha/2))

Eq(sin(theta_2/2)*cos(m/2), n_y*sin(alpha/2))

Eq(sin(p/2)*cos(theta_2/2), n_z*sin(alpha/2))

Instead, assume $\sin(p/2) \neq 0$, then we can substitute in the first equation $\cos\left(\theta_2/2\right)$ with its value computed from the last equation. We get:

In [27]:
sp.trigsimp(sp.Eq(rhs_simplified.a, q.a).subs(sp.cos(theta2 / 2), nz * sp.sin(alpha / 2) / sp.sin(p / 2)))

Eq(n_z*sin(alpha/2)/tan(p/2), cos(alpha/2))

Therefore
$$
p = \theta_1 + \theta_3 = 2 \arctan \left[n_z \tan
\left(
\frac{\alpha}{2} \right)\right]
$$

We can then deduce
$$
\begin{array}{rl}
\theta_2 & = 2 \arccos \left[ \cos \left(\frac{\alpha}{2}\right) \cos^{-1}\left(\frac{p}{2}\right) \right] \\
&= 2 \arccos \left[ \cos\left(\frac{\alpha}{2}\right) \sqrt{1 + n_z^2 \tan^2 \left(
  \frac{\alpha}{2}  \right) } \right] \\
\end{array}
$$

Using the third equation, we can finally deduce the value of $m$:
$$
m=\theta_1 - \theta_3 = 2\arccos\left[\frac{ n_y \sin\left(\frac{\alpha}{2}\right) } {\sin\left(\frac{\theta_2}{2}\right)}\right]
$$

The terms are similar to the other decompositions, XZX, YXY, ZXZ, XYX and YZY. However, for ZXZ, XYX and YZY, the $i$ term in the quaternion is positive as seen below in the YZY decomposition.

In [28]:
theta1, theta2, theta3 = sp.symbols("theta_1 theta_2 theta_3")

y1 = sp.algebras.Quaternion.from_axis_angle((0, 1, 0), theta1)
z = sp.algebras.Quaternion.from_axis_angle((0, 0, 1), theta2)
y2 = sp.algebras.Quaternion.from_axis_angle((0, 1, 0), theta3)

rhs = sp.trigsimp(sp.expand(y1 * z * y2))
rhs

cos(theta_2/2)*cos((theta_1 + theta_3)/2) + sin(theta_2/2)*sin((theta_1 - theta_3)/2)*i + sin((theta_1 + theta_3)/2)*cos(theta_2/2)*j + sin(theta_2/2)*cos((theta_1 - theta_3)/2)*k

Thus, in order to correct for the orientation of the rotation, $\theta_1$ and $\theta_3$ are swapped. 

### 3.2 Mckay Decomposition
The McKay decomposition can be shown by studying the matrix representation of a generic gate with 3 Euler angles, $\theta$, $\phi$, and $\lambda$. The $U$ gate can be seen below, which is capable of forming any unitary matrix or Bloch sphere rotation. 

$$
U(\theta,\phi, \lambda) = 
\begin{bmatrix}
 \mathrm{cos}(\theta/2) & -i e^{i\lambda}\mathrm{sin}(\theta/2) \\
-i e^{i\phi}\mathrm{sin}(\theta/2) & e^{i(\lambda+\phi)}\mathrm{cos}(\theta/2)
\end{bmatrix}
$$

This can be conveniently represented as,
$$
U(\theta,\phi, \lambda) = Rz(\phi) \cdot Rx(\theta) \cdot Rz(\lambda). 
$$

Using the identity,
$$
Rx(\theta) = Rz(-\pi/2) \cdot Rx(\pi/2) \cdot Rz(\pi - \theta) \cdot Rx(\pi/2) \cdot Rz(-\pi/2), 
$$
one can show that any unitary gate is,
$$
U(\theta,\phi, \lambda) = Rz(\phi-\pi/2) \cdot Rx(\pi/2) \cdot Rz(\pi - \theta) \cdot Rx(\pi/2) \cdot Rz(\lambda-\pi/2).
$$

### 3.3 ABA Decomposition
In a two qubit system, the unitary matrix of a controlled operation is seen below,
$$ 
U =
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & U_{00} & U_{01} \\
0 & 0 & U_{10} & U_{11}
\end{pmatrix}
$$

OpenSquirrel utilises the ABC decomposition (https://arxiv.org/abs/quant-ph/9503016) to decompose multi-qubit gates. The trick is to express a 1-qubit unitary $U$ as,
$$
U = e^{i\alpha} A \times B \times C,
$$
where gates $A$, $B$, and $C$ are chosen such that $ABC = I$. An arbitrary controlled unitary is then expressed as,
```
                                q_0: ───────■─────────■────────■─────
                                     ┌───┐┌─┴─┐┌───┐┌─┴─┐┌───┐ │Ph(α) 
                                q_1: ┤ A ├┤ X ├┤ B ├┤ X ├┤ C ├─■─────
                                     └───┘└───┘└───┘└───┘└───┘       
```
where the first qubit acts as a control for two CNOTs and a controlled phase gate with a rotation of $\alpha$. This can be reformulated as,
```
                                                          ┌───────┐
                                q_0: ───────■─────────■───┤ Rz(α) ├
                                     ┌───┐┌─┴─┐┌───┐┌─┴─┐ └─┬───┬─┘
                                q_1: ┤ A ├┤ X ├┤ B ├┤ X ├───┤ C ├──
                                     └───┘└───┘└───┘└───┘   └───┘  
```
where the last controlled gate is substituted with a single qubit Z-axis rotation of $\alpha$ degrees.

OpenSquirrel uses Z-Y-Z to construct the ABC gates. Consider,
$$
\begin{split}
        U &= e^{i\alpha} R_z(\theta_2) R_y(\theta_1) R_z(\theta_0) \\
        &= e^{i\alpha} R_z\big(\theta_2\big) R_y\big(\frac{1}{2}\theta_1\big) R_y\big(\frac{1}{2}\theta_0\big)R_z\big(\frac{1}{2}\theta_0+\frac{1}{2}\theta_2\big)R_z\big(\frac{1}{2}\theta_0-\frac{1}{2}\theta_2\big) \\
        &= e^{i\alpha} R_z\big(\theta_2\big) R_y\big(\frac{1}{2}\theta_1\big) X R_y\big(-\frac{1}{2}\theta_0\big) X X R_z\big(-\frac{1}{2}\theta_0-\frac{1}{2}\theta_2\big) X R_z\big(\frac{1}{2}\theta_0-\frac{1}{2}\theta_2\big) \\
        &= e^{i\alpha} A X B X C
\end{split},
$$
thus, 
$$
\begin{split}
A &= Rz\bigg(\theta_2 \bigg)Ry\bigg( \frac{1}{2} \theta_1 \bigg), \\
B &= Ry\bigg( - \frac{1}{2} \theta_1 \bigg)Rz\bigg( -\frac{1}{2} \theta_0 - \frac{1}{2}\theta_2 \bigg),\\
C &= Rz\bigg(\frac{1}{2} \theta_0 - \frac{1}{2} \theta_2 \bigg)
\end{split}
$$